<a href="https://colab.research.google.com/github/imamurayusuke/SD1111_colab/blob/main/SD1111_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ⬇［SD/WebUIの起動／停止］この下の再生ボタンをクリックするとSD/WebUIが起動／停止します

# @markdown > Running on local URL:  [http&#58;//127.0.0.1:7860](http://)<br>
# @markdown > Running on public URL: [https&#58;//xxxxxxxx-xxxx-xxxx.gradio.live](http://)

# @markdown 再生ボタンをクリックして7分～10分程度でこの表示が出ます。下の行のリンクをクリックするとWebUIが開きます。

import os

# RAMとVRAMの容量取得(GiB)
res = !free -m | grep 'Mem:' | awk '{print $2}'
if not res:
   raise ValueError("RAM容量を取得できません")
ram = round(float(res[0]) / 1024, 3)
res = !nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits
if not res:
   raise ValueError("VRAM容量を取得できません。GPUに接続していない可能性があります。「編集」メニューの「ノートブックの設定」で「ハードウェアアクセラレータ」を「GPU」に変更してください。")
vram = round(float(res[0]) / 1024, 3)
print(f"RAM : {ram}")
print(f"VRAM: {vram}")
del res

# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

!sudo apt autoremove
!sudo apt-get -y update
!sudo apt-get -y install python3.10
!sudo cp `which python3.10` /usr/local/bin/python

# Pythonのバージョン確認
!python -V

!wget https://bootstrap.pypa.io/get-pip.py
!sudo python get-pip.py

!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -U --pre triton
!pip install xformers==0.0.16rc425
!pip install fastapi==0.90

# SD/WebUIのセットアップ
%cd /content/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd stable-diffusion-webui/models/Stable-diffusion

# 公式学習モデルのダウンロード
!wget -c "https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors" -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.safetensors
!wget -c "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.vae.pt
!wget -c "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml" -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

# でりだモデルのダウンロード
!wget -c "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/derrida_final.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/derrida_final.ckpt
!wget -c "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/autoencoder_fix_kl-f8-trinart_characters.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/derrida_final.vae.pt

# Googleドライブ上の設定ファイルをバックアップしランタイム上へシンボリックリンク
sdpath     = "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion/"
configpath = sdpath + "config.json"
uicfgpath  = sdpath + "ui-config.json"
stylespath = sdpath + "styles.csv"

if os.path.isfile(configpath):
   !cp -v "{configpath}" "{sdpath}config_bak.json"
   !ln -s -v "{configpath}" "/content/stable-diffusion-webui/config.json"
if os.path.isfile(uicfgpath):
   !cp -v "{uicfgpath}" "{sdpath}ui-config_bak.json"
   !ln -s -v "{uicfgpath}" "/content/stable-diffusion-webui/ui-config.json"
if os.path.isfile(stylespath):
   !cp -v "{stylespath}" "{sdpath}styles_bak.csv"
   !ln -s -v "{stylespath}" "/content/stable-diffusion-webui/styles.csv"

# Googleドライブ上に画像の出力先フォルダを作成（初回のみ）、ランタイム上からそこへシンボリックリンク
outputspath = sdpath + "outputs"
logpath     = sdpath + "log"

if not os.path.exists(outputspath):
   !mkdir -p -v "{outputspath}"
!ln -s -v "{outputspath}" "/content/stable-diffusion-webui/outputs"

if not os.path.exists(logpath):
   !mkdir -p -v "{logpath}"
!ln -s -v "{logpath}" "/content/stable-diffusion-webui/log"

# ランタイム上のExtensionsフォルダにあるput extensions here.txtをGoogleドライブにコピー（初回のみ）
extpath    = sdpath + "extensions"

if not os.path.exists(extpath):
   !mv -v "/content/stable-diffusion-webui/extensions" "{sdpath}"
else:
   !mv -v "/content/stable-diffusion-webui/extensions" "/content/stable-diffusion-webui/extensions_org"

#Extensionsフォルダへランタイム上からシンボリックリンク
!ln -s -v "{extpath}" "/content/stable-diffusion-webui/"

# 必ず利用するエクステンションをダウンロード（初回のみ）
if not os.path.exists(extpath + "/stable-diffusion-webui-localization-ja_JP"):
   !git clone https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-ja_JP
else:
   %cd /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-ja_JP
   !git pull 
   %cd /content/stable-diffusion-webui/models/Stable-diffusion

# 一度起動して必要なものをインストールする
%cd /content/stable-diffusion-webui/
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py

# SD/WebUIの起動
!COMMANDLINE_ARGS="--xformers --opt-channelslast --share --enable-insecure-extension-access --ckpt /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.safetensors" REQS_FILE="requirements.txt" python launch.py


In [ ]:
#@title ⬇［設定のバックアップ］SD/WebUIを停止した後に実行してください

# @markdown 以下のファイルがGoogleドライブにバックアップされます。

# @markdown * 初回起動時に生成される設定ファイル（ui-config.json）

# @markdown * Settingsタブの「Apply settings」を初めてクリックしたときに生成される設定ファイル（config.json）

# @markdown * Stylesを初めて登録したときに生成される設定ファイル（styles.csv）

# @markdown 一度バックアップされれば、以降はSD/WebUIが起動時にGoogleドライブ上の設定ファイルを読み出し、設定を変更すればGoogleドライブ上の設定ファイルも更新されます。

# @markdown バックアップ先は「Colab Notebooks」－「Stable Diffusion」フォルダです。

# @markdown 実行後のメッセージの末尾が3行とも「～ are the same file」と出るようになれば、すべてGoogleドライブに保存されています。以降はこのコードセルを実行する必要はなくなります。
!cp -v "/content/stable-diffusion-webui/config.json" "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion"
!cp -v "/content/stable-diffusion-webui/ui-config.json" "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion"
!cp -v "/content/stable-diffusion-webui/styles.csv" "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion"

In [ ]:
#@title ⬇［再起動］SD/WebUIの再起動用コードセル

#@markdown いったん起動してから停止したSD/WebUIを再起動するときに使います。ダウンロードやインストールを省略できるため約1分で起動します。

%cd /content/stable-diffusion-webui/
!COMMANDLINE_ARGS="--xformers --opt-channelslast --share --enable-insecure-extension-access --ckpt /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.safetensors" REQS_FILE="requirements.txt" python launch.py

## このノートブックについて
- このノートブックの詳細は[imamurayusuke/SD1111\_colab \- GitHub](https://github.com/imamurayusuke/SD1111_colab)をご覧ください
- 作成にあたっては[aka\_1111\.ipynb \- Colaboratory](https://colab.research.google.com/drive/1GhvterZebPlZU4P7RcUK-U-4Y23rZpQO)を参考にさせていただきました

